In [ ]:
import aliases

In [ ]:
from everest.utilities import classtools as _classtools

In [ ]:
###############################################################################
''''''
###############################################################################


import functools as _functools
from abc import ABCMeta as _ABCMeta
import itertools as _itertools
from collections import abc as _collabc

from everest.utilities.misc import (
    TypeMap as _TypeMap,
    )
from everest.utilities import classtools as _classtools


class ChoraMeta(_ABCMeta):

    def __init__(cls, /, *args, **kwargs):
        super().__init__(*args, **kwargs)
        cls._cls_extra_init_()

    def add_child_class(cls, ACls):
        truename = ACls.__name__.strip('_')
        if ACls in cls.__mro__:
            addcls = cls
        else:
            classpath = []
            if 'classpath' in cls.__dict__:
                classpath.extend(cls.classpath)
            else:
                classpath.append(cls)
            addcls = type(
                truename,
                (ACls, cls),
                dict(classpath=tuple((*classpath, truename)))
                )
        setattr(cls, truename, addcls)


_IncMethsLike = _collabc.Iterator[tuple[type, _collabc.Callable]]


@_classtools.Diskable
@_classtools.MROClassable
class Chora(metaclass=ChoraMeta):

    @classmethod
    def child_classes(cls):
        return iter(())

    @_classtools.MROClass
    class Element:
        __slots__ = ('context', 'index')
        def __init__(self, context, index):
            self.context, self.index = context, index
        def __call__(self):
            return self.context.retrieve(self.index)

    @_classtools.Diskable
    @_classtools.MROClass
    class Incision:

        def __init__(self, context, chora):
            self.context, self.chora = context, chora
            self.register_argskwargs(context, chora)

        @property
        def choracontext(self):
            return self.context

        def retrieve(self, arg):
            return self.context.retrieve(arg)

    @classmethod
    def decorate(cls, ACls, /):
        old_init = ACls.__init__
        if not hasattr(ACls, 'chora_args'):
            def chora_args(self):
                return iter(())
            ACls.chora_args = chora_args
        if not hasattr(ACls, 'chora_kwargs'):
            def chora_kwargs(self):
                return iter(())
            ACls.chora_kwargs = chora_kwargs
        if not hasattr(ACls, 'retrieve'):
            raise TypeError(
                "Incisable must provide a callable `.retrieve` attribute."
                )
        def chora_extra_init(self, *args, **kwargs):
            old_init(self, *args, **kwargs)
            self.chora = cls(
                *self.chora_args(),
                **dict(self.chora_kwargs())
                )
        ACls.__init__ = chora_extra_init
        cls.add_defer_methods(ACls)
        return ACls

    @classmethod
    def add_defer_methods(cls, ACls, /):
        if not hasattr(ACls, 'choracontext'):
            ACls.choracontext = property(lambda x: x)
        def __getitem__(self, incisor):
            out = self.chora.__getitem__(incisor, context=self.choracontext)
            if isinstance(out, self.chora.Element):
                return out()
            return self.chora.Incision(self.choracontext, out)
        ACls.__getitem__ = __getitem__
        def __contains__(self, arg):
            return self.chora.__contains__(arg)
        ACls.__contains__ = __contains__

    @classmethod
    def _cls_extra_init_(cls, /):
        cls.incmeths = cls._get_incmeths()
        cls.add_defer_methods(cls.Incision)
        for child in cls.child_classes():
            cls.add_child_class(child)

    @classmethod
    def incision_methods(cls, /) -> _IncMethsLike:
        '''Returns acceptable incisor types and their associated getmeths.'''
        return iter(())

    @classmethod
    def priority_incision_methods(cls, /) -> _IncMethsLike:
        '''Returns like `.incision_methods` but takes priority.'''
        yield tuple, cls.incise_tuple
        yield type(Ellipsis), cls.incise_trivial

    @classmethod
    def _get_incmeths(cls, /) -> _TypeMap:
        return _TypeMap(
            _itertools.chain(
                cls.priority_incision_methods(),
                cls.incision_methods()
                ),
            )

    def __init__(self, /, *, criterion):
        self.register_argskwargs(criterion=criterion)
        if isinstance(type(criterion), type):
            criterionclass = criterion
            def criterion(arg):
                return isinstance(arg, criterionclass)
        elif not callable(criterion):
            raise TypeError(criterion)
        self.criterion = criterion

    def __getitem__(self, incisor, /, *, context):
        try:
            meth = self.incmeths[type(incisor)]
        except KeyError as exc:
            raise TypeError from exc
        return meth(self, incisor, context=context)

    def __contains__(self, arg, /):
        return self.criterion(arg)

    def incise_tuple(self, incisor, /, *, context):
        '''Captures the special behaviour implied by `context[a,b,c...]`'''
        raise TypeError("Tuple slicing not supported.")

    def incise_trivial(self, incisor=None, /, *, context):
        '''Captures the special behaviour implied by `context[...]`.'''
        return context

    def incise_strict(self, incisor, /, *, context):
        '''Captures the sense of retrieving a single item from `context`.'''
        return self.Element(context, incisor)


###############################################################################
###############################################################################

In [ ]:
###############################################################################
''''''
###############################################################################


import itertools as _itertools

from everest import utilities as _utilities

_Chora = Chora


class Sliceable(_Chora):

    @classmethod
    def _cls_extra_init_(cls, /):
        super()._cls_extra_init_()
        cls.slcmeths = dict(reversed(tuple(cls.slice_methods())))

    @classmethod
    def slice_methods(cls, /):
        yield (False, False, False), cls.incise_trivial

    def incise_slice(self, slc, /, *, context):
        return self.incise_slyce(
            _utilities.misc.slyce(slc),
            context=context,
            )

    def incise_slyce(self, slc, /, *, context):
        try:
            slcmeth = self.slcmeths[slc.hasargs]
        except KeyError:
            raise ValueError(" ".join((
                f"Object of type {type(self)}",
                "cannot be sliced with slice of",
                "start={0}, stop={1}, step={2}".format(*slc.args),
                )))
        return slcmeth(self, *slc, context=context)

    @classmethod
    def incision_methods(cls, /):
        yield slice, cls.incise_slice
        yield _utilities.misc.Slyce, cls.incise_slyce
        yield from super().incision_methods()


class _Sampling_:

    def __init__(self, *args, sampler, **kwargs):
        sampler = self.sampler = self.process_sampler(sampler)
        self.register_argskwargs(sampler=sampler)
        super().__init__(*args, **kwargs)

    @classmethod
    def process_sampler(cls, sampler):
        return sampler

    @classmethod
    def combine_samplers(self, a, b):
        '''Combines multiple samplers into one.'''
        return (a, b)

    def incise_sampler(self, sampler, /, *, context):
        '''Captures the sense of `context[::sampler]`'''
        sampler = self.combine_samplers(self.sampler, sampler)
        return super().incise_sampler(sampler, context=context)


class Sampleable(Sliceable):

    @classmethod
    def child_classes(cls):
        yield from super().child_classes()
        yield _Sampling_

    @classmethod
    def slice_methods(cls, /):
        yield (False, False, True), cls.incise_sampler_slice
        yield from super().slice_methods()

    def incise_sampler(self, sampler, /, *, context):
        '''Captures the sense of `context[::sampler]`'''
        return self.Sampling(
            *self.args,
            **(self.kwargs | dict(sampler=sampler)),
            )

    def incise_sampler_slice(self, _, __, sampler, /, *, context):
        return self.incise_sampler(sampler, context=context)


###############################################################################
###############################################################################

In [ ]:
###############################################################################
''''''
###############################################################################


import operator as _operator

from everest.utilities import classtools as _classtools

_Sampleable = Sampleable


def make_comparator_from_gt(gt):
    def comparator(a, b):
        return gt(a, b) - gt(b, a)
    return comparator


class _Delimited_:

    def __init__(self, *args, lbnd, ubnd, **kwargs):
        self.bnds = self.lbnd, self.ubnd = lbnd, ubnd
        self.register_argskwargs(lbnd=lbnd, ubnd=ubnd)
        super().__init__(*args, **kwargs)

    def slice_in_range(self, start, stop):
        comparator, (lbnd, ubnd) = self.comparator, self.bnds
        if lbnd is None or start is None:
            lcheck = True
        else:
            lcheck = comparator(start, lbnd) > -1
        if ubnd is None or stop is None:
            ucheck = True
        else:
            ucheck = comparator(stop, ubnd) < 1
        return lcheck and ucheck

    def __contains__(self, item):
        if super().__contains__(item):
            comparator = self.comparator
            return all((
                comparator(item, self.lbnd) > -1, # item >= lbnd
                comparator(item, self.ubnd) < 0, # item < ubnd
                ))


class Orderable(_Sampleable):

    @classmethod
    def child_classes(cls):
        yield from super().child_classes()
        yield _Delimited_

    @classmethod
    def slice_methods(cls, /):
        for comb in ((True, True), (True, False), (False, True)):
            yield (*comb, False), cls.incise_delimit_slice
            yield (*comb, True), cls.incise_delimit_sample
        yield from super().slice_methods()

    def __init__(self, /, *args, comparator=None, gt=_operator.gt, **kwargs):
        if comparator is None:
            self.comparator = make_comparator_from_gt(gt)
            self.register_argskwargs(gt=gt)
        else:
            self.comparator = comparator
            self.register_argskwargs(comparator=comparator)
        super().__init__(*args, **kwargs)

    def slice_in_range(self, start, stop):
        return all(
            self.__contains__(st)
            for st in (start, stop) if st is not None
            )

    def incise_delimit(self, start, stop, /, *, context):
        if not self.slice_in_range(start, stop):
            raise KeyError("Slice out of range.")
        return self.Delimited(
            *self.args,
            **(self.kwargs | dict(lbnd=start, ubnd=stop)),
            )

    def incise_delimit_slice(self, start, stop, _, /, *, context):
        return self.incise_delimit(start, stop, context=context)

    def incise_delimit_sample(self, start, stop, step, /, *, context):
        return (
            self.incise_delimit(start, stop, context=context)
            .incise_sampler(step, context=context)
            )


###############################################################################
###############################################################################

In [ ]:
###############################################################################
''''''
###############################################################################


_Orderable = Orderable


class Advanceable(_Orderable):

    @classmethod
    def add_defer_methods(cls, ACls, /):
        def __iter__(self):
            return map(self.retrieve, self.chora.__iter__())
        ACls.__iter__ = __iter__
        super().add_defer_methods(ACls)

    def __init__(self, *args, advancer, **kwargs):
        self.advancer = advancer
        self.register_argskwargs(advancer=advancer)
        super().__init__(*args, **kwargs)

    def __iter__(self):
        advancer = self.advancer
        val = yield
        while True:
            yield val
            val = advancer(val)


class Enumerable(Advanceable, Orderable.Delimited):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._iterfn = {
            (False, True): self._iter_ubnd,
            (True, False): self._iter_lbnd,
            (True, True): self._iter_dbnd,
            }[tuple(bnd is not None for bnd in self.bnds)]

    def _iter_lbnd(self):
        val, advancer = self.lbnd, self.advancer
        while True:
            yield val
            val = advancer(val)

    def _iter_ubnd(self):
        advancer = self.advancer
        val = yield
        while True:
            if comparator(val, ubnd) > -1:
                return
            yield val
            val = advancer(val)

    def _iter_dbnd(self):
        comparator, ubnd = self.comparator, self.ubnd
        for val in self._iter_lbnd():
            if comparator(val, ubnd) > -1:
                return
            yield val

    def __iter__(self):
        return self._iterfn()
        

###############################################################################
###############################################################################

In [ ]:
def myfn(*, a=None, b, c):
    return a, b, c

In [ ]:
import operator
import functools


advancer = functools.partial(operator.add, 1)


class MyChora(Enumerable):

    @classmethod
    def incision_methods(cls):
        yield int, cls.incise_strict
        yield from super().incision_methods()


@MyChora.decorate
class MyClass:

    def __init__(self, *args, **kwargs):
        self.contents = list(*args, **kwargs)

    def retrieve(self, arg):
        return self.contents[arg]

    def chora_kwargs(self):
        yield 'advancer', advancer
        yield 'criterion', int
        yield 'lbnd', 0
        yield 'ubnd', len(self.contents)


import string
myobj = MyClass(string.ascii_lowercase)
assert myobj[0] == 'a'

In [ ]:
list(myobj[1:10])

In [ ]:
myinc = myobj[1:10]

In [ ]:
type(myinc).__mro__

In [ ]:
assert type(myinc).mroclassowner is not None

In [ ]:
myinc.get_redtup()[0]

In [ ]:
myinc

In [ ]:
myinc.dump()

In [ ]:
type(myinc.chora).__mro__

In [ ]:
myinc.chora.Sampling.classpath